https://dicom-numpy.readthedocs.io/en/latest/

#### This command will unblock all files in a directory
get-childitem "C:\Users\caleb\Desktop\461P\FinalProj\Alz-Finders\preproc\002_S_0295\MP-RAGE\2006-04-1
8_08_20_30.0\S13408" | unblock-file

In [4]:
import pydicom
import dicom_numpy



In [5]:
import os

#print(os.getcwd())
#dataPath = os.getcwd() + "\..\\ADNI_Playset\\ADNIPlaySet\\ADNI"
dataPath = os.getcwd()
#print(dataPath)
#for file in os.listdir(dataPath ):
#    print(file)

In [6]:
def extract_voxel_data(list_of_dicom_files):
    datasets = [pydicom.read_file(f) for f in list_of_dicom_files]
    try:
        voxel_ndarray, ijk_to_xyz = dicom_numpy.combine_slices(datasets)
    except dicom_numpy.DicomImportException as e:
        # invalid DICOM data
        raise
    return voxel_ndarray


smallListDCM = [os.listdir(dataPath)[1] ] #this line is jank and depends on the order of the files in the directory
print(type(smallListDCM))
print(smallListDCM)
voxData = extract_voxel_data(smallListDCM)



<class 'list'>
['002_S_0295']


PermissionError: [Errno 13] Permission denied: '002_S_0295'